# Ray Tune + (light) Train su Fashion-MNIST (CPU)

> Prima di eseguire: `pip install -U ray "ray[tune,train]" torch torchvision`

In [ ]:

import os, ray, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

class FashionCNN(nn.Module):
    def __init__(self, hidden_units=600):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(64*6*6, hidden_units)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(hidden_units, 120)
        self.fc3 = nn.Linear(120, 10)
    def forward(self, x):
        x = self.layer1(x); x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x); x = self.drop(x); x = self.fc2(x); x = self.fc3(x)
        return x

def train_fashion_mnist(config):
    tfm = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    ds = datasets.FashionMNIST(root=".data", train=True, download=True, transform=tfm)
    dl = DataLoader(ds, batch_size=int(config["batch_size"]), shuffle=True)
    model = FashionCNN(hidden_units=int(config["hidden_units"]))
    crit = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=float(config["lr"]))
    epochs = 3  # light
    for epoch in range(epochs):
        model.train(); running = 0.0
        for images, labels in dl:
            opt.zero_grad()
            loss = crit(model(images), labels)
            loss.backward(); opt.step()
            running += loss.item()
        train.report({"loss": running/len(dl), "epoch": epoch+1})

search_space = {
    "lr": tune.loguniform(1e-3, 1e-1),
    "batch_size": tune.choice([32, 64, 128]),
    "hidden_units": tune.choice([400, 600, 800])
}

ray.shutdown(); ray.init()
scheduler = ASHAScheduler(max_t=3, grace_period=1, reduction_factor=2)
reporter = CLIReporter(metric_columns=["loss","training_iteration"])
result = tune.run(
    train_fashion_mnist,
    resources_per_trial={"cpu":1, "gpu":0},
    config=search_space, metric="loss", mode="min",
    num_samples=4, scheduler=scheduler, progress_reporter=reporter
)
best_trial = result.get_best_trial("loss", "min", "last")
print("Best config:", best_trial.config)
ray.shutdown()
